## Automating Table Shortcuts in Microsoft Fabric for Unified Data Strategy in OneLake
**by Jesus Lopez Martin - https://www.syntax.es - November 2024**

This Python script demonstrates how to automate the creation of shortcuts for tables between a source Workspace and Lakehouse and a destination Workspace and Lakehouse in Microsoft Fabric. It aligns with the "single source of truth" strategy in OneLake and supports the Medallion architecture, simplifying data access and management across workspaces. By leveraging the Microsoft Fabric APIs and the semantic-link-labs library, this approach helps streamline data integration and reduce redundancy.

**Key Benefits:**
- Centralized Data Access: This script supports the "single source of truth" strategy by enabling seamless access to tables across workspaces and Lakehouses without data duplication.
- Simplified Management: Automates the creation of shortcuts, reducing manual effort and operational overhead.
- Medallion Architecture Support: Facilitates the creation of Bronze, Silver, and Gold layers in Microsoft Fabric by linking data across Lakehouses.


In [ ]:
%pip install semantic-link-labs --q

In [ ]:
import sempy_labs.lakehouse as lakehouse

# Step 1: Get all tables from the source Lakehouse
source_workspace = "TEST Source Workspace"
source_lakehouse = "Source_Lakehouse"
destination_workspace = "TEST Destination Workspace"
destination_lakehouse = "Destination_Lakehouse"

# Retrieve tables as a DataFrame
df_tables = lakehouse.get_lakehouse_tables(
    lakehouse=source_lakehouse, 
    workspace=source_workspace
)

# Step 2: Loop through the DataFrame and create shortcuts for each table
for table_name in df_tables['Table Name']:  # Iterate directly over the 'name' column
    lakehouse.create_shortcut_onelake(
        table_name=table_name,
        source_lakehouse=source_lakehouse,
        source_workspace=source_workspace,
        destination_lakehouse=destination_lakehouse,
        destination_workspace=destination_workspace,
        shortcut_name=table_name  # Using the same name for the shortcut
    )
    print(f'Successfully created shortcut for table: {table_name}')
